In [10]:
import syft as sy
sy.requires(">=0.8,<0.8.1")

✅ The installed version of syft==0.8.1b2 matches the requirement >=0.8 and the requirement <0.8.1


In [11]:
node = sy.orchestra.launch(name="test-domain-1", port=8080, dev_mode=True, reset=True)

Starting test-domain-1 server on 0.0.0.0:8080

SQLite Store Path:
!open file:///tmp/7bca415d13ed1ec841f0d0aede098dbb.sqlite

Stopping process on port: 8080
Process 125585 terminated.


INFO:     Started server process [125800]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)


INFO:     127.0.0.1:57062 - "GET /api/v1/new/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:57062 - "POST /api/v1/new/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:57062 - "GET /api/v1/new/api?verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f HTTP/1.1" 200 OK
INFO:     127.0.0.1:45010 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:55152 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:55164 - "GET /api/v1/new/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:32928 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:32930 - "GET /api/v1/new/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:55792 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:55808 - "GET /api/v1/new/api?verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f HTTP/1.1" 200 OK
INFO:     127.0.0.1:43946 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:39728 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
IN

In [12]:
domain_client = node.login(email="info@openmined.org", password="changethis")

In [13]:
from typing import List, Dict, Any, Optional

In [14]:
class RepeatedCallPolicy(sy.CustomOutputPolicy):
    n_calls: int = 0
    downloadable_output_args: List[str] = []
    state: Dict[Any, Any] = {}

    def __init__(self, n_calls=1, downloadable_output_args: List[str] = None):
        self.downloadable_output_args = downloadable_output_args if downloadable_output_args is not None else []
        self.n_calls = n_calls + 1
        self.state = {"counts": 0}

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, context, outputs):
        output_dict = {}
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                output_dict[output_arg] = outputs[output_arg]

            self.state["counts"] += 1
        else:
            return None

        return output_dict

In [15]:
policy = RepeatedCallPolicy(n_calls=1, downloadable_output_args=['y'])

In [16]:
policy.n_calls

2

In [17]:
policy.downloadable_output_args

['y']

In [18]:
policy.init_kwargs

{'n_calls': 1, 'downloadable_output_args': ['y']}

In [19]:
print(policy.init_kwargs)
a_obj = sy.ActionObject.from_obj({'y': [1,2,3]})
x = policy.apply_output(context=None, outputs=a_obj)
x

{'n_calls': 1, 'downloadable_output_args': ['y']}


,key,type,id
0,y,"Pointer:\n[1, 2, 3]",56586fba9e104dd58db6fac6c18da1e3


In [20]:
policy.n_calls

2

In [22]:
import numpy as np
x = np.array([1,2,3])
x_pointer = sy.ActionObject.from_obj(x)
x_pointer

```python
Pointer
```
array([1, 2, 3])

In [23]:
domain_client.api.services.action.save(x_pointer)

<class 'syft.service.response.SyftSuccess'>: <class 'syft.service.action.numpy.NumpyArrayObject'> saved

In [25]:
x_pointer.id

<UID: fdeb2ca43e38462a9e40538f6e91629a>

In [26]:
@sy.syft_function(
    input_policy=sy.ExactMatch(x=x_pointer),
    output_policy=RepeatedCallPolicy(n_calls=10, downloadable_output_args=['y']),
)
def func(x):
    return {"y": x+1}

In [27]:
@sy.syft_function(input_policy=sy.ExactMatch(x=x_pointer),
                  output_policy=sy.SingleExecutionExactOutput())
def train_mlp(x):
    return x

In [28]:
domain_client.api.services.code.request_code_execution(func)

```python
class Request:
  id: str = bc4a0a5b30f3447abb3e71ab2cea56ad
  requesting_user_verify_key: str = aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f
  approving_user_verify_key: str = None
  request_time: str = 2023-05-16 02:33:48
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  request_hash: str = "f7d8a968812ec5cdc5b664215138feb3ccd85897a267c108d9f6c64e83b25688"
  changes: str = [syft.service.request.request.UserCodeStatusChange]

```

In [29]:
from syft import MessageStatus
messages = domain_client.api.services.messages.get_all_for_status(MessageStatus.UNDELIVERED)
messages

,type,id,subject,status,created_at,linked_obj
0,syft.service.message.messages.Message,06d139fd743b43e9a562f12d72e523f0,Approval Request,MessageStatus.UNDELIVERED,2023-05-16 02:33:48,<<class 'syft.service.request.request.Request'...


In [30]:
from syft.service.request.request import ObjectMutation, UserCodeStatusChange, Request
func = None
request = None
for message in messages:
    req = message.linked_obj.resolve
    if isinstance(req, Request):
        for change in req.changes:
            if isinstance(change, UserCodeStatusChange):
                user_code = change.linked_obj.resolve
                if "func" in user_code.service_func_name:
                    func = user_code
                    request = req

In [31]:
func

```python
class UserCode:
  id: str = 55587187baaa4ea696cc84160d7cdff0
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  user_verify_key: str = aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f
  raw_code: str = "@sy.syft_function(
    input_policy=sy.ExactMatch(x=x_pointer),
    output_policy=RepeatedCallPolicy(n_calls=10, downloadable_output_args=['y']),
)
def func(x):
    return {"y": x+1}
"
  input_policy_type: str = <class 'syft.service.policy.policy.ExactMatch'>
  input_policy_init_kwargs: str = {NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): {'x': <UID: fdeb2ca43e38462a9e40538f6e91629a>}}
  input_policy_state: str = b''
  output_policy_type: str = syft.service.policy.policy.UserPolicy
  output_policy_init_kwargs: str = {'n_calls': 10, 'downloadable_output_args': ['y']}
  output_policy_state: str = b''
  parsed_code: str = "def user_func_func_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f_d7f8db0378b044d89c2d33a89c7eeba45f24ae57f515a61a0dd3027bbf674862(x):

    def func(x):
        return {'y': x + 1}
    result = func(x=x)
    return result"
  service_func_name: str = "func"
  unique_func_name: str = "user_func_func_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f_d7f8db0378b044d89c2d33a89c7eeba45f24ae57f515a61a0dd3027bbf674862"
  user_unique_func_name: str = "user_func_func_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f"
  code_hash: str = "d7f8db0378b044d89c2d33a89c7eeba45f24ae57f515a61a0dd3027bbf674862"
  signature: str = (x)
  status: str = {NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): <UserCodeStatus.SUBMITTED: 'submitted'>}
  input_kwargs: str = ['x']
  enclave_metadata: str = None

```

In [32]:
domain_client.api.services.messages

,type,id,subject,status,created_at,linked_obj
0,syft.service.message.messages.Message,06d139fd743b43e9a562f12d72e523f0,Approval Request,MessageStatus.UNDELIVERED,2023-05-16 02:33:48,<<class 'syft.service.request.request.Request'...


In [33]:
for message in messages:
    domain_client.api.services.messages.mark_as_delivered(message.id)

In [34]:
domain_client.api.services.messages

,type,id,subject,status,created_at,linked_obj
0,syft.service.message.messages.Message,06d139fd743b43e9a562f12d72e523f0,Approval Request,MessageStatus.DELIVERED,2023-05-16 02:33:48,<<class 'syft.service.request.request.Request'...


In [35]:
request

```python
class Request:
  id: str = bc4a0a5b30f3447abb3e71ab2cea56ad
  requesting_user_verify_key: str = aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f
  approving_user_verify_key: str = None
  request_time: str = 2023-05-16 02:33:48
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  request_hash: str = "f7d8a968812ec5cdc5b664215138feb3ccd85897a267c108d9f6c64e83b25688"
  changes: str = [syft.service.request.request.UserCodeStatusChange]

```

In [22]:
request.approve()

<class 'syft.service.response.SyftSuccess'>: Request 283518a713ae4328bfae78874a858719 changes applied

In [37]:
request.changes

,type,id
0,syft.service.request.request.UserCodeStatusChange,9893dfb47bc3467ea357d3b56b0b9cdc


In [38]:
request.changes[0]

```python
class UserCodeStatusChange:
  id: str = 9893dfb47bc3467ea357d3b56b0b9cdc
  linked_obj: str = <<class 'syft.service.code.user_code.UserCode'>: 55587187baaa4ea696cc84160d7cdff0@<Node: 7bca415d13ed1ec841f0d0aede098dbb>
  value: str = UserCodeStatus.EXECUTE
  match_type: str = True

```

In [39]:
print(f"{x_pointer = }")
result = func.unsafe_function(x=x_pointer)
result

x_pointer = Pointer:
[1 2 3]


{'y': array([2, 3, 4])}

In [40]:
final_result = request.accept_by_depositing_result(result) 
final_result

<class 'syft.service.response.SyftSuccess'>: Request bc4a0a5b30f3447abb3e71ab2cea56ad changes applied

In [41]:
res = domain_client.api.services.code.func(x=x_pointer)
res

Pointer:
{'y': array([2, 3, 4])}

In [42]:
assert (res["y"] == np.array([2, 3, 4])).all()

In [43]:
assert set(res.syft_action_data.keys()) == set(list("y"))

In [44]:
domain_client.api.services.code.get_all()[0].output_policy

```python
class RepeatedCallPolicy_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f_ba90eef16c2b39292a2ee07f1aefc157216b84487b26c32a1512d114c8ee7c69:
  id: str = da752724b0fa48fca22c740f66341efa
  init_kwargs: str = {}
  output_history: str = []
  output_kwargs: str = []
  node_uid: str = None
  n_calls: str = 11
  downloadable_output_args: str = ['y']
  state: str = {'counts': 2}

```

In [45]:
domain_client.api.services.policy.get_all()

[]

In [46]:
output_policy = domain_client.api.services.policy.get_all()
output_policy

[]

In [33]:
node.land()

Stopping test-domain-1
